# Mission to Mars Scrapping

Site to explore: <a href="https://redplanetscience.com/">Redplanet Science</a>

In [2]:
# Importing Dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests
from urllib.request import urlopen as uReq

In [3]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Jose.Vazquez\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# 1. Latest Mars news extraction

In [5]:
# Defining URL to scrape

url = "https://redplanetscience.com/"
browser.visit(url)

In [6]:
html = browser.html
soup = bs(html, 'html.parser')

title_list = []
art_list = []

title = soup.find_all('div', class_='content_title')
article = soup.find_all('div', class_='article_teaser_body')

list_e =[title, article]

for titl, art in zip(title, article):
    print(f'Title: {titl.text}')
    print(f'Article extract: {art.text}')
    print('----------------------------------')
    title_list.append(titl.text)
    art_list.append(art.text)

Title: NASA's Perseverance Rover Is Midway to Mars 
Article extract: Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.
----------------------------------
Title: From JPL's Mailroom to Mars and Beyond
Article extract: Bill Allen has thrived as the mechanical systems design lead for three Mars rover missions, but he got his start as a teenager sorting letters for the NASA center.
----------------------------------
Title: NASA to Hold Mars 2020 Perseverance Rover Launch Briefing
Article extract: Learn more about the agency's next Red Planet mission during a live event on June 17.
----------------------------------
Title: NASA's Mars Perseverance Rover Passes Flight Readiness Review
Article extract: ​The agency's Mars 2020 mission has one more big prelaunch review – the Launch Readiness Review, on July 27.
----------------------------------
Title: NASA to Broadcast Mars 

# 2. Featured Image URL build

In [28]:
space_img_url = "https://spaceimages-mars.com/"
browser.visit(space_img_url)

In [8]:
html_img = browser.html
soup_img = bs(html_img, 'html.parser')

header_img = soup_img.find('img', class_="headerimage")['src']

featured_image_url = space_img_url+header_img

print(featured_image_url)


https://spaceimages-mars.com/image/featured/mars3.jpg


# 3. Mars facts table extraction

In [27]:
facts_url = "https://galaxyfacts-mars.com/"
browser.visit(facts_url)

In [10]:
tables = pd.read_html(facts_url)
display(tables)
tables[1]

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


# 4. Mars Images extraction

In [46]:
mult_img_url = ("https://marshemispheres.com/")
browser.visit(mult_img_url)

In [179]:
mult_img_html = browser.html
soup = bs(mult_img_html, 'html.parser')

art_list = soup.find_all('a', class_='itemLink product-item')
h3_list = soup.find_all('h3')
mars_img_list = []


for n, h3 in enumerate(h3_list):

    try:                   
        browser.links.find_by_partial_text(h3.text).click()
        crrnt_page = browser.html
        #print(crrnt_page)
        page_soup = bs(crrnt_page, 'html.parser')
        img_download = page_soup.find_all('div', class_='downloads')
        img_href = img_download[0].a['href']
        #print(crrnt_page)
        # print(page_soup)
        # print('---------------------------------------')
        # print(img_download)
        # print('---------------------------------------')
        # print(img_href)
        # print('---------------------------------------')
        print(h3.text)
        mars_img_list.append({'title '+str(n):h3.text, 'img url '+str(n):mult_img_url+img_href})
        print('------------------')
        browser.visit(mult_img_url)

    
    except:
        
        browser.visit(mult_img_url)
        print("Scraping finished going back to first page") 
        break

print(mars_img_list)


Cerberus Hemisphere Enhanced
------------------
Schiaparelli Hemisphere Enhanced
------------------
Syrtis Major Hemisphere Enhanced
------------------
Valles Marineris Hemisphere Enhanced
------------------
Scraping finished going back to first page
[{'title 0': 'Cerberus Hemisphere Enhanced', 'img url 0': 'https://marshemispheres.com/images/full.jpg'}, {'title 1': 'Schiaparelli Hemisphere Enhanced', 'img url 1': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'}, {'title 2': 'Syrtis Major Hemisphere Enhanced', 'img url 2': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'}, {'title 3': 'Valles Marineris Hemisphere Enhanced', 'img url 3': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]


In [108]:
print(art_list)

[<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>, <a class="itemLink product-item" href="schiaparelli.html"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="schiaparelli.html">
<h3>Schiaparelli Hemisphere Enhanced</h3>
</a>, <a class="itemLink product-item" href="syrtis.html"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>, <a class="itemLink product-item" href="syrtis.html">
<h3>Syrtis Major Hemisphere Enhanced</h3>
</a>, <a class="itemLink product-item" href="valles.html"><img alt="Val